# AI Research Assistant - Testing Notebook

This notebook provides interactive tests for the AI Research Assistant backend components.

## Setup
Make sure you have:
1. Installed dependencies: `pip install -r requirements.txt`
2. Set up your `.env` file with `HF_API_TOKEN`
3. Added `jupyter` to your environment: `pip install jupyter`


In [ ]:
# Add parent directory to path for imports
import sys
sys.path.insert(0, '..')

# Load environment variables
from dotenv import load_dotenv
load_dotenv('../.env')

print("Environment loaded!")


## 1. Test Configuration


In [ ]:
from app.config import get_settings

settings = get_settings()
print(f"App Name: {settings.app_name}")
print(f"LLM Model: {settings.hf_model_id}")
print(f"Embedding Model: {settings.embedding_model}")
print(f"Chunk Size: {settings.chunk_size}")
print(f"HF Token Set: {'Yes' if settings.hf_api_token else 'No'}")


## 2. Test LLM Client


In [ ]:
from app.llm import HFInferenceClient

llm_client = HFInferenceClient()
print(f"LLM Client initialized with model: {llm_client.model_id}")


In [ ]:
# Test text generation
response = await llm_client.generate(
    "What is machine learning? Answer in one sentence.",
    max_new_tokens=100,
    temperature=0.7
)
print("Generated response:")
print(response)


In [ ]:
# Test embeddings
test_texts = ["Hello world", "Machine learning is fascinating"]
embeddings = llm_client.get_embeddings(test_texts)
print(f"Generated {len(embeddings)} embeddings")
print(f"Embedding dimension: {len(embeddings[0])}")


## 3. Test RAG Retriever


In [ ]:
from app.retriever import RAGRetriever

retriever = RAGRetriever()
print("RAG Retriever initialized")


In [ ]:
# Ingest sample documents
sample_docs = [
    {
        "content": """
        Transformers are a type of neural network architecture that has revolutionized 
        natural language processing. They use self-attention mechanisms to process 
        sequential data in parallel, unlike RNNs which process sequentially. 
        The transformer architecture was introduced in the paper 'Attention Is All You Need' 
        by Vaswani et al. in 2017. Key components include multi-head attention, 
        positional encodings, and feed-forward layers.
        """,
        "source": "transformers_overview.txt"
    },
    {
        "content": """
        RAG (Retrieval-Augmented Generation) is a technique that combines information 
        retrieval with text generation. It retrieves relevant documents from a knowledge 
        base and uses them as context for generating responses. This helps reduce 
        hallucinations and grounds the model's outputs in factual information.
        RAG systems typically use dense retrieval with embeddings and vector similarity search.
        """,
        "source": "rag_explained.txt"
    },
    {
        "content": """
        Fine-tuning is the process of taking a pre-trained model and training it further 
        on a specific dataset for a particular task. This leverages the general knowledge 
        learned during pre-training while adapting the model to new domains or tasks.
        Common fine-tuning approaches include full fine-tuning, LoRA, and prompt tuning.
        """,
        "source": "fine_tuning_guide.txt"
    }
]

for doc in sample_docs:
    doc_id, chunks = await retriever.ingest(doc["content"], doc["source"])
    print(f"Ingested '{doc['source']}': {chunks} chunks (ID: {doc_id[:8]}...)")


In [ ]:
# Test retrieval
query = "How do transformers work?"
results = await retriever.retrieve(query, top_k=3)

print(f"Query: {query}\n")
print(f"Found {len(results)} relevant documents:\n")
for i, (doc, score) in enumerate(results, 1):
    print(f"{i}. Source: {doc.source} (Score: {score:.4f})")
    print(f"   Content: {doc.content[:100]}...")
    print()


## 4. Test Research Planner


In [ ]:
from app.agents import ResearchPlanner

planner = ResearchPlanner()

# Test query decomposition
complex_query = "What are the main differences between transformers and RNNs, and when should I use each?"
plan = await planner.create_research_plan(complex_query)

print(f"Original Query: {plan['original_query']}\n")
print(f"Strategy: {plan['strategy']}")
print(f"Estimated Steps: {plan['estimated_steps']}\n")
print("Sub-questions:")
for i, sq in enumerate(plan['sub_questions'], 1):
    print(f"  {i}. {sq}")


## 5. Test Reasoning Agent (Full Pipeline)


In [ ]:
from app.agents import ReasoningAgent

# Use the retriever we already populated with documents
reasoning_agent = ReasoningAgent(retriever)
print("Reasoning Agent initialized")


In [ ]:
# Test single question answering
question = "What is RAG and why is it useful?"
answer, sources = await reasoning_agent.answer_single_question(question, max_sources=3)

print(f"Question: {question}\n")
print(f"Answer:\n{answer}\n")
print(f"Sources used: {len(sources)}")
for s in sources:
    print(f"  - {s['source']} (relevance: {s['relevance_score']:.4f})")


In [ ]:
# Test full multi-step reasoning
complex_question = "How can I improve my NLP model's performance using modern techniques?"

result = await reasoning_agent.reason_and_answer(
    query=complex_question,
    max_sources=5,
    use_decomposition=True
)

print(f"Question: {complex_question}\n")
print("=" * 50)
print("REASONING STEPS:")
print("=" * 50)
for step in result['reasoning_steps']:
    print(f"  → {step}")

print("\n" + "=" * 50)
print("FINAL ANSWER:")
print("=" * 50)
print(result['answer'])

print(f"\nConfidence: {result['confidence']:.2%}")
print(f"Sources used: {len(result['sources'])}")
